# TF.Serving Communication
Given a model running somewhere (or on docker at localhost:8501, called `mymodel`), construct a tf.Example instance with the tf.train.Feature values. Send it to the model server through a POST request for predictions.

In [1]:
import requests
import base64

import tensorflow as tf

In [7]:
# https://www.tensorflow.org/tutorials/load_data/tfrecord
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [25]:
payload = {
    "pickup_community_area": 6,
    "fare": 9.05,
    "trip_start_month": 6,
    "trip_start_hour": 15,
    "trip_start_day": 7,
    "trip_start_timestamp": 1371913200,
    "pickup_latitude": 41.9442266,
    "pickup_longitude": -87.65599818,
    "dropoff_latitude": 41.9442266,
    "dropoff_longitude": -87.65599818,
    "trip_miles": 2.7,
    "pickup_census_tract": 0,
    "dropoff_census_tract": 0,
    "payment_type": "Cash",
    "company": "",
    "trip_seconds": 540,
    "tips": 0.0,
    "dropoff_community_area": 6
}

for k in payload.keys():
    v = payload[k]
    if type(v) is str:
        payload[k] = _bytes_feature(str.encode(v))
    elif type(v) is float:
        payload[k] = _float_feature(v)
    else:
        payload[k] = _int64_feature(v)


In [26]:
example_proto = tf.train.Example(features=tf.train.Features(feature=payload))
example_proto

features {
  feature {
    key: "company"
    value {
      bytes_list {
        value: ""
      }
    }
  }
  feature {
    key: "dropoff_census_tract"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "dropoff_community_area"
    value {
      int64_list {
        value: 6
      }
    }
  }
  feature {
    key: "dropoff_latitude"
    value {
      float_list {
        value: 41.9442253112793
      }
    }
  }
  feature {
    key: "dropoff_longitude"
    value {
      float_list {
        value: -87.65599822998047
      }
    }
  }
  feature {
    key: "fare"
    value {
      float_list {
        value: 9.050000190734863
      }
    }
  }
  feature {
    key: "payment_type"
    value {
      bytes_list {
        value: "Cash"
      }
    }
  }
  feature {
    key: "pickup_census_tract"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "pickup_community_area"
    value {
      int64_list {
        value: 6
     

In [27]:
resp = requests.post('http://localhost:8501/v1/models/mymodel:predict', json={
        'instances': [
            {'examples': {'b64': base64.b64encode(example_proto.SerializeToString()).decode('utf-8')}}
        ]})
resp.json()

{'predictions': [0.0969476178]}

In [ ]:
# {'predictions': [0.0969476178]} < 0.5 or other threshold. This guy is not a big tipper probably.